# Bevezetés a dekorátorok használatába

A Python függvények fontos tulajdonsága, hogy szerepelhetnek, mint függvény paraméter, lehetne visszatérési értékben (return) és váltózónak is átadhatóak értékként. Ezt használjuk ki a függvényeint "becsomagolásakor"

Ezen felül a függvényeknek lehetnek alfüggvényei is, amik csak az adott függvény kontextusán belül léteznek. 



In [20]:
# függvény változó értékeként

def hello(name):
    return(f"Hello {name}!")

def szepnapot(name):
    return(f"Szép napot {name}!")

def udv(name):
    return(f"Üdvözöllek, kedves {name}!")

koszontes = hello          # a "koszontes" változó értékként kapja a "hello" függvényt
print(koszontes('Árpád'))  # és innentől "koszontes()" formában hívható

koszontes = szepnapot
print(koszontes('Árpád'))

koszontes = udv
print(koszontes('Árpád'))



Hello Árpád!
Szép napot Árpád!
Üdvözöllek, kedves Árpád!


In [23]:
# függvény visszatérési értékként

def szulo(num):      # külső függvény definíció
    def gyerek1():   # egyik belső függvény
        return("Én vagyok az első gyerek.")

    def gyerek2():   # másik belső függvény
        return("Én vagyok a második gyerek.")

    if num == 1:    
        return gyerek1   # függvényt (referenciát) adunk vissza értékként

    if num == 2:
        return gyerek2   # függvényt (referenciát) adunk vissza értékként

    return None

elso = szulo(1)     # a "szulo" függvény hívása után a változóban egy függvény lesz
masodik = szulo(2)

print(elso())       # és azt a függvényt meg is lehet hívni
print(masodik())

Én vagyok az első gyerek.
Én vagyok a második gyerek.


# És akkor most csomagoljunk

Az előzőeket alkalmazva összeállítunk egy csomagoló (wrapper) függvényt, ami
* valami feladatot elvégez a ténylegesen meghívandó függvény előtt
* meghívja magát a függvényt
* és valami feladatot elvégez utána is

(később kevésbé lesz ködös, hogy mire jó ez)

A ``fontos_fuggveny()``-t csomagoljuk be és csomagolással együtt adjuk értékül a ``dolgozz`` változónak.
A ``dolgozz()`` meghívásakor lefut mindhárom feladatrész.

Szóval lényegében **a dekorátor becsomagol egy függvényt úgy, hogy megváltoztatja a viselkedését**

In [24]:

def elso_dekoratorom(func):
    def wrapper():
        print("Ez történik a tényleges függvényhívás előtt")
        func()
        print("Ez történik a tényleges függvényhívás után")
    return wrapper

def fontos_fuggveny():
    print("Most csinálom a fontos dolgokat")

dolgozz = elso_dekoratorom(fontos_fuggveny)

dolgozz()

Ez történik a tényleges függvényhívás előtt
Most csinálom a fontos dolgokat
Ez történik a tényleges függvényhívás után


Még egy példa: időkorlát felállítása.

A ``csak_nappal()`` függvény a ``wrapper()``-be csomagolja a ``kiabalj()``-t, ennek hatására csak 7 és 22 között fog kiabálni, egyébként nem. A lényeg az, hogy ehhez magát a lényegi funkcionalitást tartalmazó függvényt nem kell módosítani. 

In [31]:

from datetime import datetime

def csak_nappal(func):
    def wrapper():
        if 7 <= datetime.now().hour <= 22:
            func()
        else:
            pass # Pssszt, a szomszédok alszanak!
    return wrapper



def kiabalj():
    print("Juhúúúúú!")

kiabalj = csak_nappal(kiabalj)

kiabalj()


## A Pythonos szintaxis

Az eddigiek jók voltak, de legyen akkor már szép is. Erre való a ``@`` operátor (pie operator). Lényegében ugyanazt csináljuk vele, mint a fenti példákban, csak tömörebben és olvashatóbban. 

A ``@`` úgy használódik, hogy a ``kiabalj = csak_nappal(kiabalj)`` helyett csak odaírjuk a függvénydefiníció elé a dekorátor nevét egy ``@`` jellel. Nézzük tehát a fenti példát még egyszer, csak most az új szintexissal.


In [3]:
from datetime import datetime

def csak_nappal(func):
    def wrapper():
        if 7 <= datetime.now().hour <= 22:
            func()
        else:
            pass # Pssszt, a szomszédok alszanak!
    return wrapper


@csak_nappal
def kiabalj():
    print("Juhúúúúú!")

kiabalj()

Juhúúúúú!


Ezen felül a dekorátorokat legikább modulokba kiszervezve szokás használni, szóval a dekorátor kód általában nem is jelenik meg a tényleges funkció kódja mellett.
Így kényelmesen újrafelhasználhatóvá válnak a saját dekorátoraink.

## Paraméterek átadása

ha olyan függvényhez írnánk dekorátort, ami paramétert is kezel, akkor ezt figyelembe kell venni a ``wrapper()`` függvény összeállításánál. Erre jó a jó öreg *tuple unpacking* és *dict unpacking* a ``\*args`` és ``\*\*kwargs`` használatával. 

In [5]:
# ha nem kezeljük az átadandó paramétereket, abból van:
# több paramétert adunk át a wrapper-nek, mint kéne. 
# nulla darabot vár, de egyet adunk. 

def csinald_ketszer(func):
    def wrapper():
        func()
        func()

    return wrapper

@csinald_ketszer
def haliho(nev):
    print(f"Halihó, {nev}!")


haliho('Barnabás')


TypeError: wrapper() takes 0 positional arguments but 1 was given

A ``*args, **kwargs`` úgy bontódik ki, hogy veszi a függvények átadott összes pozicionális és összes keyword paramétert és odaadja a függvénynek egy tuple és egy dict formájában. Ide most elég annyit tudni, hogy ez így anyit jelent, hogy **az összes mindenféle paraméter, amivel a függvényt meghívtad**. 

Ez persze eléggé pongyola fogalmazás. A további részletekért keress arra, hogy *tuple unpacking* meg *dict unpacking*

In [13]:
# paraméter átadások lekezelve

def csinald_ketszer(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)

    return wrapper

@csinald_ketszer
def haliho(nev):
    print(f"Halihó, {nev}!")


haliho('Barnabás')

Halihó, Barnabás!
Halihó, Barnabás!


## Paraméter visszaadás a dekorált függvényekből

Az eddigi próbálkozásainkban nem vártunk visszaadott értéket a dekorált függvényeinktől, csak "csináltattunk" vele dolgokat. Leginkább kiíratást. Ha a függvényünk ``return`` által visszaad valamit, amire szükségünk is van, akkor gondoskodni kell róla, hogy a csomagolt függény is vissza tudja adni ugyanazt. Az előző példában a ``wrapper`` függvény végén nincs return, csak szmán véget ér, ami miatt a visszatérési értéke mindig ``None`` lesz. Változtassunk ezen. 

In [12]:
# paraméter átadások lekezelve
# ÉS eredmény visszaadás is kezelve

def csinald_ketszer(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs) # a második futás eredményét visszaadjuk return-nal
                                     # ez most itt épp megfelel a célnak, de nem mindig
                                     # ennyire egyszerű.
    return wrapper

@csinald_ketszer
def haliho(nev):
    print(f"Most futok '{nev}' paraméterrel.")
    return f"Halihó, {nev}!"

h = haliho('Barnabás') # így már lesz itt is visszatérési érték, ami a h-ba kerül
print(h)

Most futok 'Barnabás' paraméterrel.
Most futok 'Barnabás' paraméterrel.
Halihó, Barnabás!


A Python objektumok mindenféle infót tudnak magukról futásidőben is. Például a saját nevüket meg címüket. Ezt éppen kissé megkeverjük a dekorátor használatával, ami amúgy logikus is. Az alábbi példában látható, hogy az imént definiált ``haliho`` függvény úgy tudja, hogy az ő neve valójában ``wrapper``, ami amúgy meg is felel a valóságnak. 

In [17]:
help(haliho)

haliho.__doc__


Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



Az életben általában mégis inkább azt szeretnénk, hogy a dekorált függvényeink is a saját nevükön hivatkozzanak magukra. Erre használható a ``@functools.wraps`` dekorátor, ami felülírja a dekorált függvény infóit az eredeti függvény adatai szerint. 

In [18]:
# állítsuk helyre a dekorált függvény infóit - egy dekorátorral
import functools


def csinald_ketszer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs) # a második futás eredményét visszaadjuk return-nal
                                     # ez most itt épp megfelel a célnak, de nem mindig
                                     # ennyire egyszerű.
    return wrapper

@csinald_ketszer
def haliho(nev):
    print(f"Most futok '{nev}' paraméterrel.")
    return f"Halihó, {nev}!"

h = haliho('Barnabás') # így már lesz itt is visszatérési érték, ami a h-ba kerül
print(h)

Most futok 'Barnabás' paraméterrel.
Most futok 'Barnabás' paraméterrel.
Halihó, Barnabás!


In [19]:
# Most már a "haliho" függvény úgy tudja magáról, hogy ő a "haliho"
help(haliho)

haliho.__doc__

Help on function haliho in module __main__:

haliho(nev)



## Két példakód


Függvény futásidejének mérése: az ``ido_elpocsekolasa`` függvényünket dekoráljuk a ``timer`` dekorátorunkkal úgy, hogy a dekorátort is dekoráljuk a ``functools.wraps`` dekorátorral. 

1. Start időt rögzítjük változóban
2. Lefuttatjuk az eredeti függvényt, a kiemetét rögzítjük a ``value`` változóba mentjük
3. Stop időt rögzítjük változóban
4. Futásidőt kiszámítjuk az elmentett idők különbségéből
5. A ``wrapper``-ből kilépve visszaadjuk az eredeti függvény elmentett kimenetét


In [8]:
import functools
import time

def timer(func):
    """Megmérjük a dekorált függvény futásidejét"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1.
        value = func(*args, **kwargs)       # 2.
        end_time = time.perf_counter()      # 3.
        run_time = end_time - start_time    # 4.
        print(f"{func.__name__!r} függvény futási ideje {run_time:.4f}s volt")
        return value                        # 5. 
    return wrapper_timer

@timer
def ido_elpocsekolasa(ennyiszer):
    for _ in range(ennyiszer):
        sum([i**2 for i in range(10000)])


ido_elpocsekolasa(354)

'ido_elpocsekolasa' függvény futási ideje 1.0363s volt


Debug  információk kiírása

1. listába gyűjtjük az eredeti paramétereket a ``repr()`` által stringgé alakítva
2. listába gyűjtjük a keyword paramétereket *kulcs = érték* formában. A ``!r`` a ``repr()`` helyett szerepel a string összeállításnál.
3. összesítjük a fenti két listát
4. kiírjuk, hogy pontosan hogyan fogjuk meghívni a függvényünket
5. lefuttatjuk az eredeti függvényt és a kimenetét elmentjük a ``value`` változóba
6. kiírjuk a kiemeti értéket, szépen formázva, megint a ``!r`` segítségével
7. visszaadjuk az eredeti függvény elmentett kimenetét a hívónak




In [7]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"{func.__name__} hívása ezekkel a paraméterekkel: ({signature})")           # 4
        value = func(*args, **kwargs)                            # 5
        print(f"{func.__name__!r}  {value!r} értékkel tért vissza")           # 6
        return value                                             # 7
    return wrapper_debug

@debug
def udvozlo_szoveg(nev, kor=None):
    if kor is None:
        return f"Hali {nev}!"
    else:
        return f"Tyűha, {nev}! Már {kor} vagy? Jól megnőttél!"
    

udv = udvozlo_szoveg('Jocó', kor=13)
print(udv)



udvozlo_szoveg hívása ezekkel a paraméterekkel: ('Jocó', kor=13)
'udvozlo_szoveg'  'Tyűha, Jocó! Már 13 vagy? Jól megnőttél!' értékkel tért vissza
Tyűha, Jocó! Már 13 vagy? Jól megnőttél!


Ezeket a dekorátorokat a beépített fügvényekre is használhatjuk. Mivel a függvény definíciója nem itt van, a ``@`` operátor helyett vissza kell térni a legelején használt megoldásra

In [3]:
import math

# itt dekoráljuk a math.factorial-a a saját debug dekorátorunkkal
math.factorial = debug(math.factorial)

# ebben a függvénybe felhasználjuk a dekorált factorial-t
def e_kozelito_erteke(menetek=18):
    return sum(1 / math.factorial(n) for n in range(menetek))

e_kozelito_erteke(menetek = 14)


factorial hívása ezekkel a paraméterekkel: (0)
'factorial'  1 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (1)
'factorial'  1 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (2)
'factorial'  2 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (3)
'factorial'  6 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (4)
'factorial'  24 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (5)
'factorial'  120 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (6)
'factorial'  720 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (7)
'factorial'  5040 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (8)
'factorial'  40320 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (9)
'factorial'  362880 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (10)
'factorial'  3628800 értékkel tért vissza
factorial hívása ezekkel a paraméterekkel: (11)
'factorial'  39

2.7182818284467594

Egy dekorátorral nem is muszáj feltétlenül csomagolni egy függvényt. Lehet, hogy arra van csak szükség, hogy regisztráljuk, mint pl az alábbi egyszerű plugin rendszerben. 

Függvényeket regisztrálunk egy listába, amiből aztán véletlenszerűen választunk egyet.

In [5]:
import random

PLUGINS = dict()

def register(func):
    """Függvény regisztrálása a plugin tárba"""
    PLUGINS[func.__name__] = func
    return func

@register
def hello_mondo(nev):
    return f"Helló {nev}!"

@register
def jonapot_mondo(nev):
    return f"Jó napot kívánok, {nev}!"

@register
def szevasz_mondo(nev):
    return f"Szevasz, {nev}, mizu?"


def veletlen_koszones(nev):
    udvozlo, udvozlo_fuggveny = random.choice(list(PLUGINS.items()))
    print(f"Köszontés a {udvozlo!r} használatával:")
    return udvozlo_fuggveny(nev)

print(veletlen_koszones('János'))
print(veletlen_koszones('Pál'))
print(veletlen_koszones('Imre'))



Köszontés a 'hello_mondo' használatával:
Helló János!
Köszontés a 'jonapot_mondo' használatával:
Jó napot kívánok, Pál!
Köszontés a 'jonapot_mondo' használatával:
Jó napot kívánok, Imre!


## Dekorátor extra - különkiadás

Dekorátorokat mindenféle kreatív kombinációban lehet alkalmazni az eddigieken túl is. Íme pár lehetőség:

Használhatunk dekorátort osztályok metódusaira. Erre van is a gyári készletben egy nagyon hasznos: ``property``, aminek a segítségével függvény által előállított értéket jeleníthetünk meg úgy, mintha osztályváltozó lenne, ezáltal pedig lehetnek csak olvasható, csak írható, vagy épp értékvalidált osztályváltozóink. Erről lesz külön írás. 

In [11]:
# ha külön modulba raktuk volna őket, akkor egy import kellene,
# de most itt vannak fentebb definiálva

# from decorators import debug, timer

class IdoElnyelo:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def ido_elnyelo(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

i = IdoElnyelo(16)

i.ido_elnyelo(327)

__init__ hívása ezekkel a paraméterekkel: (<__main__.IdoElnyelo object at 0x0000022D44055340>, 16)
'__init__'  None értékkel tért vissza
'ido_elnyelo' függvény futási ideje 0.0013s volt


Kis kitekintésképpen: a fenti metódusban látható ``_`` a for ciklus változójának a helyén egy érdekes állat. Alapból a ``_`` felveszi az utolsó művelet értékét, de amúgy sima változóként üzeel. Olyan helyen szokás használni, ahogy maga a válzotó érdektelen, nem akarjuk az értékét felhasználni, de mégis valaminek muszáj ott lenni. Mint itt a for ciklusnál. 

Szokás még olyan helyen használni aláhúzást, ahol Python kulcsszót szeretnénk használni változónévnek. Csak biggyesszük mögé és jó lesz. 
Olyat nem mondhatunk, hogy ``class = 16``, vagy ``def = "az ami az "``. Úgy viszont lehet, hogy ``class_ = 16``, vagy ``def_ = "az ami az "``.



## Dekorátor alkalmazható teljes osztályra is,
 ilyenre is van gyári példa, a ``@singleton``, ami azt biztosítja, hogy egy osztályból egyetlenegy példány létezhessen egyszerre. Az osztálydekorátornál fontos, hogy ez a metódusokat nem dekorálja. Itt is érvényes, hogy a ``@decorator`` valójában csak egy rövidítés arra, hogy ``Osztaly = dekorator(Osztaly)``.

Dekorátorokat egymásba is lehet ágyazni úgy, hogy simán egymás alá felsoroljuk őket:

In [14]:
@csinald_ketszer
@debug
def hello(nev):
    print(f"Helló, kedves {nev}!")

hello('Aladár')

hello hívása ezekkel a paraméterekkel: ('Aladár')
Helló, kedves Aladár!
'hello'  None értékkel tért vissza
hello hívása ezekkel a paraméterekkel: ('Aladár')
Helló, kedves Aladár!
'hello'  None értékkel tért vissza


Nyilván a dekorátorok - mivel függvények - ugyanúgy kaphatnak paramétereket, mint bárki más. De még inkább, függvények visszatérési értéke lehet függvény.

In [17]:
import functools

def csinald_sokszor(ennyiszer):
    def ismetlo_dekorator(func):
        @functools.wraps(func)
        def wrapper_ismetlo(*args, **kwargs):
            for _ in range(ennyiszer):
                value = func(*args, **kwargs)
            return value
        return wrapper_ismetlo
    return ismetlo_dekorator

# dekoráljuk most ezzel az előző függvényünket
@csinald_sokszor(ennyiszer=4)
def hello(nev):
    print(f"Helló, kedves {nev}!")

hello('Aladár')


Helló, kedves Aladár!
Helló, kedves Aladár!
Helló, kedves Aladár!
Helló, kedves Aladár!


Állapottartó logikát is megvalósíthatunk dekorátorral. Az alábbi példa a dekorált függvényünk meghívásait tartja számon.

Itt az állapot megtartását - a hívások számát - a csomagoló függvény .num_calls attribútumában tároljuk. Ha jól értem, a függvény maga, mint objektum, kap egy attribútumot, amit aztán megőriz. (PEP 232 - Function attributes - ennek még majd utána kell járni)


In [19]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Eddig {wrapper_count_calls.num_calls} alkalommal hívtuk meg a {func.__name__!r} függvényt")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def mondd_hello():
    print("Helló!")

mondd_hello()

mondd_hello()

mondd_hello()



Eddig 1 alkalommal hívtuk meg a 'mondd_hello' függvényt
Helló!
Eddig 2 alkalommal hívtuk meg a 'mondd_hello' függvényt
Helló!
Eddig 3 alkalommal hívtuk meg a 'mondd_hello' függvényt
Helló!


Osztályokat is használhatunk dekorátorként. Ilyenkor a dekoráló függvény szerepét az osztály ``.__call__()`` metódusa veszi fel. (Egy osztály akkor hívható, ha van neki ``.__call__()`` metódusa). Ennek segítségével az adott osztályból létrejött példány is hívható, mint egy függvény. Valójában egy dekorátor osztály létrehozásához ennyi a minimum. 

In [18]:

class Szamlalo:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):  # attól válok az osztály hívhatóvá, hogy van neki ilyenje. 
        self.count +=1
        print(f"A számláló {self.count} értéken áll.")

szamlalo = Szamlalo() # itt példányosítjuk az osztályt

# és aztán hívogatjuk párszor
szamlalo()
szamlalo()
szamlalo()

A számláló 1 értéken áll.
A számláló 2 értéken áll.
A számláló 3 értéken áll.


És akkor ebből így lesz dekorátor

In [22]:
import functools

class HivasSzamlalo:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.hivasok_szama = 0

    def __call__(self, *args, **kwargs):
        self.hivasok_szama +=1
        print(f"{self.func.__name__!r} függvényt {self.hivasok_szama} alkalommal hívtad.")
        return self.func(*args, **kwargs)


@HivasSzamlalo
def mondd_hello():
    print("Helló!")

mondd_hello()

mondd_hello()

mondd_hello()


'mondd_hello' függvényt 1 alkalommal hívtad.
Helló!
'mondd_hello' függvényt 2 alkalommal hívtad.
Helló!
'mondd_hello' függvényt 3 alkalommal hívtad.
Helló!


## Singleton osztály

Singleton osztályhoz - ahogy egy osztályból egyszerre csak egyetlen példány létezhet - a következőképp állíthatunk elő dekorátort. Itt nem függvényt, hanem egy teljes osztályt dekorálunk. Ez a kód pont ugyanazt a mintát követi, mint a függvénydekorátoraink korábban, csak itt ``func`` helyett ``cls`` van, jelezve, hogy nem függvényt szeretnénk dekorálni, hanem Class-t. (Mondjuk mivel konktér tipusmegkötés nincs a kódban, nagyon sokat azért nem számít. )

Annyi a trükk, amit szintén használtunk már korábban, hogy egy függvényattribútumba eltároljuk az első létrejött példányt és a további hívások esetén ugyanazt adjuk vissza.

A kimenetben látható, hogy a két legyártott példány azonosítóját lekérdezve ugyanazt kapjuk. Valamint, mikor az ``is`` operátorral összehasonlítjuk őket, akkor egyezőnek találtatnak.

In [24]:
import functools 

def singleton(cls):
    """Biztosítja, hogy egy osztályból egyetlen példány lézezzen max."""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:                     # ha az attribútumot még nem állítottuk be
            wrapper_singleton.instance = cls(*args, **kwargs)  # akkor létrehozzuk az első példányt
        return wrapper_singleton.instance                      # és vissza is adjuk. 
    wrapper_singleton.instance = None         # itt definiálunk függvény attribútumot
    return wrapper_singleton

# a létező legegyszerűbb Class
@singleton
class AzEgyetlen:
    pass

# példányosítjuk kétszer
elso_peldany = AzEgyetlen()
masodik_peldany = AzEgyetlen()

# és lecsekkoljuk, hogy különböznek-e az aonosítóik
print(id(elso_peldany))
print(id(masodik_peldany))

# valamint
print( elso_peldany is masodik_peldany)

# és látjuk, hogy a kettő ugyanaz


2393438035056
2393438035056
True


## Adatok cache-elése dekorátorban

Itt ugyanazt a mintát használjuk, mint korábban a ``count_calls`` dekorátornál, ahol egy függvény attribútum segítségével alkottunk állapottartó szerkezetet. Itt most a ``wrapper_cache_retval`` függvényünknek adunk egy ``wrapper_cache_retval.cache`` attribútumot, ami egy dict lesz. A függvényünk hívásakor az összes átadott paraméterből létrehozunk egy kulcsot, ez lesz a ``cache_key``, ezt használjuk a dict-ünkben kulcsként. Ha a kulcs még nem szerepel a dictben, akkor értékként hozzáadjuk az eredeti függvényünk (``func()``) kimenetét. Ha már benne van, akkor simán  csak visszaadjuk a cache-elt értéket. 

Persze az itt szereplő ``print``-ek a működéshez feleslegesek. Sőt, a korábbi ``count_calls`` dekorátorral helyettesíteni is lehet többé-kevésbé.

In [16]:
import functools

def cache_retval(func):
    @functools.wraps(func)
    def wrapper_cache_retval(*args, **kwargs):
        cache_key =  args + tuple(kwargs.items())
        if not cache_key in wrapper_cache_retval.cache:
            wrapper_cache_retval.cache[cache_key] = func(*args, **kwargs)
            print(f"Új érték tárolása {func.__name__!r} függvény számára a követkető paraméterekhez: {cache_key!r}. Letárolt érték: {wrapper_cache_retval.cache[cache_key]!r}")
        else:  # ez az ág csak a demonstrááció miatt kell
            print(f"Érték kiszolgálása cache-ből a {func.__name__!r} függvény számára a követkető paraméterekhez: {cache_key!r}. Letárolt érték: {wrapper_cache_retval.cache[cache_key]!r}")
           
        return wrapper_cache_retval.cache[cache_key]

    wrapper_cache_retval.cache = dict()
    return wrapper_cache_retval

# dekoráljuk a függvényünket
@cache_retval
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

#teszteljük
print('Első menet:')
fibonacci(10)
print('\nMásodik menet:')
fibonacci(10)



Első menet:
Új érték tárolása 'fibonacci' függvény számára a követkető paraméterekhez: (1,). Letárolt érték: 1
Új érték tárolása 'fibonacci' függvény számára a követkető paraméterekhez: (0,). Letárolt érték: 0
Új érték tárolása 'fibonacci' függvény számára a követkető paraméterekhez: (2,). Letárolt érték: 1
Érték kiszolgálása cache-ből a 'fibonacci' függvény számára a követkető paraméterekhez: (1,). Letárolt érték: 1
Új érték tárolása 'fibonacci' függvény számára a követkető paraméterekhez: (3,). Letárolt érték: 2
Érték kiszolgálása cache-ből a 'fibonacci' függvény számára a követkető paraméterekhez: (2,). Letárolt érték: 1
Új érték tárolása 'fibonacci' függvény számára a követkető paraméterekhez: (4,). Letárolt érték: 3
Érték kiszolgálása cache-ből a 'fibonacci' függvény számára a követkető paraméterekhez: (3,). Letárolt érték: 2
Új érték tárolása 'fibonacci' függvény számára a követkető paraméterekhez: (5,). Letárolt érték: 5
Érték kiszolgálása cache-ből a 'fibonacci' függvény számár

55